In [1]:
import os
from vllm import LLM, SamplingParams
from vllm.steer_vectors.request import SteerVectorRequest, VectorConfig
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/")
os.environ["VLLM_USE_V1"] = "0"
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
llm = LLM(model="/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/", enable_steer_vector=True, enforce_eager=True, tensor_parallel_size=1)

example = "Please reason step by step, and put your final answer within \\boxed{}.\nUser: " + "2 + 3 = ?" + "\nAssistant: <think>"
example_answers = llm.generate(
    example,
    SamplingParams(
        temperature=0,
        max_tokens=4096,
        skip_special_tokens=False,
    ),
)
print("=====Baseline=====")
print(example_answers[0].outputs[0].text)
print("Length: ", len(tokenizer.tokenize(example_answers[0].outputs[0].text, add_special_tokens=True)))


target_suffix = "ĊĊ"
# 获取分词器的完整词汇表，这是一个 {token: id} 形式的字典
vocab = tokenizer.get_vocab()

# 查找所有符合条件的 token 和 id
matching_tokens_ids = [
    token_id
    for token, token_id in vocab.items()
    if isinstance(token, str) and token.endswith(target_suffix)
]

sv_request = SteerVectorRequest(
    steer_vector_name="complex_control",
    steer_vector_id=4,
    vector_configs=[
        # # 理解阶段的向量
        VectorConfig(
            path="execution_avg_vector.gguf",
            scale=1.0,
            target_layers=[20], 
            generate_trigger_tokens=matching_tokens_ids, 
            algorithm="direct",
            normalize=False
        ),
         VectorConfig(
            path="reflection_avg_vector.gguf",
            scale=-1.0,
            target_layers=[20], 
            generate_trigger_tokens=matching_tokens_ids, 
            algorithm="direct",
            normalize=False
        ),
         VectorConfig(
            path="transition_avg_vector.gguf",
            scale=-1.0,
            target_layers=[20], 
            generate_trigger_tokens=matching_tokens_ids, 
            algorithm="direct",
            normalize=False
        ),
    ],
    debug=False,
    conflict_resolution="sequential"
)
output = llm.generate(example, SamplingParams(
        temperature=0,
        max_tokens=4096,
        skip_special_tokens=False,
    ), steer_vector_request=sv_request)
print("=====Seal=====")
print(output[0].outputs[0].text)
print("Seal: ", len(tokenizer.tokenize(output[0].outputs[0].text, add_special_tokens=True)))

INFO 07-12 02:49:55 [__init__.py:244] Automatically detected platform cuda.
INFO 07-12 02:50:08 [config.py:841] This model supports multiple tasks: {'classify', 'reward', 'generate', 'embed'}. Defaulting to 'generate'.
INFO 07-12 02:50:08 [config.py:1472] Using max model len 131072
WARNING 07-12 02:50:08 [arg_utils.py:1577] The model has a long context length (131072). This may causeOOM during the initial memory profiling phase, or result in low performance due to small KV cache size. Consider setting --max-model-len to a smaller value.
WARNING 07-12 02:50:09 [cuda.py:102] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 07-12 02:50:09 [llm_engine.py:232] Initializing a V0 LLM engine (v0.1.dev7499+g2a4b294) with config: model='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B/', speculative_config=None, tokenizer='/data/zju-46/shenyl/hf/model/deepseek-ai/DeepSeek-R1-Distill-Qw

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 07-12 02:50:12 [default_loader.py:272] Loading weights took 1.10 seconds
INFO 07-12 02:50:12 [model_runner.py:1255] Model loading took 3.3461 GiB and 1.255380 seconds
INFO 07-12 02:50:16 [worker.py:295] Memory profiling takes 3.94 seconds
INFO 07-12 02:50:16 [worker.py:295] the current vLLM instance can use total_gpu_memory (47.44GiB) x gpu_memory_utilization (0.90) = 42.69GiB
INFO 07-12 02:50:16 [worker.py:295] model weights take 3.35GiB; non_torch_memory takes 0.06GiB; PyTorch activation peak memory takes 8.07GiB; the rest of the memory reserved for KV Cache is 31.22GiB.
INFO 07-12 02:50:17 [executor_base.py:115] # cuda blocks: 73063, # CPU blocks: 9362
INFO 07-12 02:50:17 [executor_base.py:120] Maximum concurrency for 131072 tokens per request: 8.92x
INFO 07-12 02:50:20 [llm_engine.py:430] init engine (profile, create kv cache, warmup model) took 7.73 seconds


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

=====Baseline=====

Okay, so I need to figure out what 2 plus 3 is. Hmm, let me think about this. I remember from school that when you add numbers, you combine them to get a total. So, if I have two apples and someone gives me three more apples, how many apples do I have in total?

Wait, maybe I should visualize this. If I have two blocks and I add three more blocks, how many blocks do I have? Let me count them: one, two, three, four. So, that's four blocks in total. Does that make sense? Yeah, that seems right.

Alternatively, I can think about it on a number line. Starting at 2, if I move three units to the right, I land on 5. So, 2 plus 3 equals 5. That also checks out.

Is there another way to verify this? Maybe using objects. If I have two coins and someone gives me three more coins, how many coins do I have? Counting them: one, two, three, four, five. So, again, that's five coins. That seems consistent.

I guess another way to look at it is by using addition tables or flashcards.

Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%| | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, 

=====Seal=====

Okay, so I need to figure out what 2 plus 3 is. Hmm, let me think about this. I remember from school that when you add numbers, you combine them to get a total. So, if I have two apples and someone gives me three more apples, how many apples do I have in total?

 I can visualize this. If I have two apples, and then I get three more, I can count them out. Let me count: one, two, three, four. Wait, that doesn't seem right. No, that's not how addition works. Maybe I should use my fingers to count. If I hold up two fingers and then add three more, how many do I have?

 1, 2, 3, 4. Hmm, that's four. So, 2 plus 3 equals 4. That makes sense because when you add two numbers, you're combining their quantities. So, 2 and 3 together make 4. I think that's correct.

  I can also think about it in terms of money. If I have two one-dollar bills and someone gives me three more, how much money do I have? Two plus three is five dollars. That seems right. So, 2 plus 3 is definitely 5.

 